In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pypdf
!pip install python-dotenv

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [ ]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/Data/").load_data()

In [ ]:
from llama_index.prompts.prompts import SimpleInputPrompt

# Defined a detailed system prompt tailored for constitutional law and case law chatbot
system_prompt = (
    "Welcome to AIJusTEch Pakistan."
    "Your AI legal expert is here to assist you with constitutional law and case law queries. "
    "Provide your legal questions, and the AI will deliver predictions and analyses based on constitutional principles and relevant case law. "
    "Cite relevant constitutional clauses and precedents for increased credibility. "
    "Your responses will be tailored to the constitutional and precedential context of your queries."
)


# Defined a query wrapper prompt to be used for user queries
query_wrapper_prompt = SimpleInputPrompt("Given the constitutional data reference and precedent case lists on user query:  {query_str}.What can be the decision according to constitution of Pakistan and provide some precedent cases related to this?"
                                        "End with the line: Thanks for consulting with AIJusTEch Pakistan, your legal Partner."
                                        )

In [ ]:
!huggingface-cli login

In [ ]:

import torch


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [ ]:

from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
vecindex = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
#import torch

# Save the entire model
#torch.save(llm, 'llm_model.pth')  # Model saving code

# Move the saved model to the Colab content folder for easier access
#!mv llm_model.pth /content/Data/


In [ ]:
conversation_history = []

while True:
    user_query = input("User: ")  # Get user input
    if user_query.lower() in ['exit', 'quit']:
        break  # Exit the loop if the user wants to end the conversation

    # Add user query to conversation history
    conversation_history.append(f"User: {user_query}")

    # Process the retrieved documents with your existing code
    query_engine = vecindex.as_query_engine(streaming=True)

    # Assuming your query engine supports using relevant documents
    response = query_engine.query("\n".join(conversation_history))


    response.print_response_stream()

    # Add AI response to conversation history
    conversation_history.append(f"AIJusTech: {response}")



In [ ]:
# Save configuration
llm.config.save_pretrained('/content/Data/llm_config')

# Save model weights
llm.save_pretrained('/content/Data/llm_weights')


**NGROK THING TRYING**

In [ ]:
!pip install flask
!pip install flask-ngrok




In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

conversation_history = []

app = Flask(__name__)
run_with_ngrok(app)  # Integrate ngrok into Flask to expose the app

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    user_query = request.form['user_query']

    # Add user query to conversation history
    conversation_history.append(f"User: {user_query}")

    # Process the retrieved documents with your existing code
    query_engine = vecindex.as_query_engine()

    # Assuming your query engine supports using relevant documents
    response = query_engine.query("\n".join(conversation_history))

    # Add AI response to conversation history
    conversation_history.append(f"AIJusTech: {response}")

    # Return the response to the frontend
    return render_template('index.html', user_query=user_query, ai_response=response)


if __name__ == '__main__':
    app.run()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


In [ ]:
!./ngrok authtoken 2YlFhOAi2x55W4IQmjw5CVA2g4B_7iNXHecvuaTSoPcSXtrxH


In [ ]:
!pkill ngrok


In [ ]:


# Example user query
user_query = "Are there instances where an individual has faced candidacy rejection in an election due to being classified under Schedule 4 of the Anti-Terrorism Act, resulting in the possible dismissal of their nominee papers?. What does constitution say about this matter?  "


# Process the retrieved documents with your existing code
query_engine = vecindex.as_query_engine()

# Assuming your query engine supports using relevant documents
response = query_engine.query(user_query)

# Convert the response to a string
response_str = str(response)

# Write the string to a file
with open('response.txt', 'w') as file:
    file.write(response_str)



In [ ]:
print(response)

In [ ]:
!pip install pyngrok


In [ ]:
print(conversation_history)

In [ ]:

while True:
  user_query=input()
  # Process the retrieved documents with your existing code
  query_engine = index.as_query_engine(streaming=True)

  # Assuming your query engine supports using relevant documents
  response = query_engine.query(user_query)
  response.print_response_stream()

**CONVERSATION BUFFER MEMORY TRYING**


In [ ]:
# Initialize a list to store conversation history
conversation_history = []

# Start the conversation loop
while True:
    # Get user query
    user_query = input("User: ")

    # Add user query to the conversation history
    conversation_history.append({"role": "user", "text": user_query})

    # Use the conversation history in your query engine
    query_engine = index.as_query_engine(streaming=True)

    # Combine user queries into a single string for querying
    full_user_query = " ".join(item["text"] for item in conversation_history if item["role"] == "user")

    # Query with the combined user queries
    response = query_engine.query(full_user_query)

    # Add the model's response to the conversation history
    conversation_history.append({"role": "ai", "text": response})

    # Check if the user is asking about the conversation history
    if "history" in user_query.lower():
        # Print the user's previous queries and chatbot's responses
        for item in conversation_history:
            if item["role"] == "user":
                print("User: ", item["text"])
            elif item["role"] == "ai":
                print("AI: ", item["text"])

    # Print the model's response
    else:
        print("AI: ", response)


In [ ]:
#from IPython.display import set_matplotlib_formats

# Increase the maximum number of lines displayed in the output cell
# You can adjust the number based on your needsbox_inches": "tight"}')
#get_ipython().run_line_magic('config', 'InlineBackend.figure_format = "retina"'
#get_ipython().run_line_magic('config', 'InlineBackend.print_figure_kwargs = {"b)


In [ ]:
step = 1000  # Adjust the step size as needed
for i in range(0, len(response_str), step):
    print(response_str[i:i+step])


In [ ]:


# Example user query
user_query = "Kane a christian is accused of blasphemy on the ground of the evidence in the form of a video. "


# Process the retrieved documents with your existing code
query_engine = index.as_query_engine(streaming=True)

# Assuming your query engine supports using relevant documents
response = query_engine.query(user_query)
response.print_response_stream()

# Convert the response to a string
response_str = str(response)

# Write the string to a file
with open('response.txt', 'w') as file:
    file.write(response_str)



In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.schema import HumanMessage, AIMessage

# Instantiate ConversationBufferMemory with existing chat history
past_messages = [
    HumanMessage("Welcome to AIJusTEch Pakistan."),
    AIMessage("Your AI legal expert is here to assist you with constitutional law and case law queries."),
]

memory = ConversationBufferMemory(chat_history=past_messages)

# Instantiate ConversationChain with the model and memory
chain = ConversationChain(llm=llm, memory=memory)

# Main conversation loop
while True:
    # Get user query
    user_query = input("User: ")

    # Add the user query to the conversation memory
    memory.add_message(HumanMessage(user_query))

    # Use the conversation history in your query engine
    query_engine = index.as_query_engine(streaming=True)
    response = query_engine.query(memory.get_full_history())

    # Print or handle the model's response
    print("AI: ", response)

    # Add the model's response to the conversation memory
    memory.add_message(AIMessage(response.print_response_stream()))


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install langchain==0.0.208

In [ ]:
from langchain.runnable import Runnable
